# Supervised Learning In Action
By Mahfuz Miah, November 2019

## 1. Synopsis

Data taken from: 

Lead Testing in School Drinking Water Sampling and Results: Most Recently Reported Beginning 2016
Link: https://healthdata.gov/dataset/lead-testing-school-drinking-water-sampling-and-results-most-recently-reported-beginning

Combined with:
NYS Education Department Data
Link: https://data.nysed.gov/downloads.php
SRC2018: This database contains accountability, annual assessment, and staff data for the state as well as by county, Need to Resource Capacity group, district, public school, and charter school.
STUDED2018: This database contains free and reduced-price lunch, average class size, student attendance and suspensions, staff counts, teacher attendance, and teacher turnover data.
ENROLL2018: This database contains school, district, county, and statewide enrollment by grade, race/ethnicity, gender, and other groups.

Can join tables on BEDS (Basic Educational Data System (BEDS) code) or name of school. Theoretically.




## 2. Background


## 3. Understanding and Preparing the Data Set


In [239]:
#Libraries:
# import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import statistics
from pandas import DataFrame
import datetime
import seaborn as sns
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn import linear_model
from fractions import Fraction
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import normaltest

# set up the figure size

%matplotlib inline

In [193]:
# #Load the data:
# from meza import io, process as pr, convert as cv
# from io import open

# enroll_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/enrollment_2018/ENROLL2018.mdb'
# studed_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/studed2018/STUDED2018.mdb'
# src_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/SRC2018/SRC2018_20190627.mdb'
# #
# enroll_data = io.read(enroll_path)

# #
# studed_data = io.read(studed_path) 

# #
# src_data = io.read(src_path) 


In [194]:
# with open(enroll_path) as f:
#     records = io.read_mdb(f, encoding='latin-1')

In [195]:
# next(enroll_data)

In [196]:
# enroll_data = pr.prepend(enroll_data, row)

In [197]:
# enroll_data, result = pr.detect_types(enroll_data)
# {t['id']: t['type'] for t in result['types']}

In [198]:
# enroll_data

In [199]:
# print(next(enroll_data))

In [200]:
# print(next(studed_data))

In [201]:
# print(next(src_data))

In [202]:
# from io import open
# from meza import io

# #records1 = io.read(enroll_path)
# #records2 = io.read('path/to/file.xls')

# with open(enroll_path, encoding='utf-8') as f:
#     records3 = io.read(f, ext='mdb')

# records3

In [203]:
# import pandas_access as mdb  # Link: https://github.com/jbn/pandas_access

# # Listing the tables.
# for tbl in mdb.list_tables(src_path):
#     print(tbl)

In [204]:
# # Read a small table.
# df = mdb.read_table(src_path, "ACC EM ELP")

In [205]:
# to_pandas_schema(read_schema(rdb_file, schema_encoding),kwargs.pop('implicit_string', True))

In [206]:
# # Read a huge table.
# accumulator = []
# for chunk in mdb.read_table(src_path, "ACC EM ELP", chunksize=10000):
#     accumulator.append(f(chunk))

In [207]:
# Set up the data
# Link: https://healthdata.gov/dataset/lead-testing-school-drinking-water-sampling-and-results-most-recently-reported-beginning
# Lead Testing in School Drinking Water Sampling and Results: Most Recently Reported Beginning 2016

def setdata(filename):
    df = pd.read_csv(filename)
    
    # Strip and make column names lowercase so that they are easy to manage. 
    df.columns = df.columns.str.strip().str.lower()

    # Replace characters or remove spaces and make all words lowercase.
    df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_').str.replace('(', '').str.replace(')', '')

    return df

In [208]:
# define all paths to necessary csv/txt files
lead_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/Lead_Testing.csv'
elem_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/Elementary.txt'
hs_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/HighSchool.txt'
dem_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/DemographicFactors.txt'
lunch_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/FreeReducedPriceLunch.txt'
ins_path = '/Users/Mahfuz-Tamim/Thinkful/Supervised Learning/Capstone/InstitutionGrouping.txt'

In [209]:
# set up all the dataframes from all the csv/txt files
lead_df = setdata(lead_path)
elem_df = setdata(elem_path)
hs_df = setdata(hs_path)
dem_df = setdata(dem_path)
lunch_df = setdata(lunch_path)
ins_df = setdata(ins_path)

# make a list of all the dataframes
list_dfs = [lead_df,elem_df,hs_df,dem_df,lunch_df,ins_df]

# make a list of all the names of the dataframes in string format
str_list_dfs = ['lead_df','elem_df','hs_df','dem_df','lunch_df','ins_df']

In [210]:
lead_df.head()

,school_district,school,county,type_of_organization,number_of_outlets,any_buildings_with_lead-free_plumbing?,previously_sampled_outlets,outlets_waiver_requested,waivers_granted,outlets_sampled_after_regulation,...,school_website,beds_code,school_street,school_city,school_state,school_zip_code,date_sampling_updated,date_results_updated,county_location,location
0,BOCES No District Name,SULLIVAN BOCES,Sullivan,BOCES School,NaN,No,NaN,NaN,NaN,NaN,...,NaN,599000000000,6 WIERK AVE,LIBERTY,NY,12754,NaN,NaN,"(41.705166, -74.711705)","6 WIERK AVE\nLIBERTY, NY 12754\n(41.795377, -7..."
1,BOCES No District Name,WSWHE BOCES MEYERS EDUCATIONAL CENTER MAIN BUI...,Washington,BOCES School,NaN,No,NaN,NaN,NaN,NaN,...,NaN,649000000006,15 HENNING ROAD,SARATOGA SPRINGS,NY,12866,NaN,11/29/2018,"(43.289513, -73.454761)","15 HENNING ROAD\nSARATOGA SPRINGS, NY 12866\n(..."
2,HAMBURG CENTRAL SCHOOL,HAMBURG SHS,Erie,Public School,104.0,No,0.0,0.0,0.0,104.0,...,hamburg central school district,141601060007,LEGION DR,HAMBURG,NY,14075,12/02/2016,11/07/2016,"(42.888143, -78.795544)","LEGION DR\nHAMBURG, NY 14075"
3,LYONS CENTRAL SCHOOL,LYONS ELEMENTARY SCHOOL,Wayne,Public School,29.0,No,0.0,0.0,0.0,0.0,...,www.lyonscsd.org,650501040001,98 WILLIAMS ST,LYONS,NY,14489,12/03/2016,12/03/2016,"(43.144336, -77.117995)","98 WILLIAMS ST\nLYONS, NY 14489\n(43.065833, -..."
4,MEDINA CENTRAL SCHOOL,OAK ORCHARD SCHOOL,Orleans,Public School,115.0,No,0.0,0.0,0.0,115.0,...,medinacsd.org,450801060002,W OAK ORCHARD ST,MEDINA,NY,14103,11/22/2016,09/28/2018,"(43.248394, -78.218438)","W OAK ORCHARD ST\nMEDINA, NY 14103"


In [253]:
elem_df

,entity_cd,entity_name,year,subject,subgroup_name,core_cohort,core_index,weighted_cohort,weighted_index,composite_level,override,override_weighted,override_core
0,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,All Students,7998,84.9,9112,74.5,1,NaN,NaN,NaN
1,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Students with Disabilities,1178,54.4,1447,44.3,2,NaN,NaN,NaN
2,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,American Indian or Alaska Native,50,70,66,53,1,NaN,NaN,NaN
3,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Asian or Native Hawaiian/Other Pacific Islander,756,106.7,767,105.1,2,NaN,NaN,NaN
4,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Black or African American,3771,63.3,4306,55.4,1,NaN,NaN,NaN
5,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Hispanic or Latino,1470,78.8,1680,69,1,NaN,NaN,NaN
6,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,White,1485,132.1,1781,110.1,2,NaN,NaN,NaN
7,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Multiracial,496,93.5,554,83.8,2,NaN,NaN,NaN
8,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,English Language Learners,1123,54.3,1179,51.7,2,NaN,NaN,NaN
9,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Economically Disadvantaged,6029,66.6,6797,59,1,NaN,NaN,NaN


In [211]:
elem_df[elem_df.subgroup_name == "All Students"].sort_values(by = "core_index")

,entity_cd,entity_name,year,subject,subgroup_name,core_cohort,core_index,weighted_cohort,weighted_index,composite_level,override,override_weighted,override_core
31214,512300010009,OGDENSBURG FREE ACADEMY,2018,Combined,All Students,564,100,596,94.6,2,NaN,NaN,NaN
17729,321100010567,LINDEN TREE ELEMENTARY SCHOOL,2018,Combined,All Students,377,100,377,100,2,NaN,NaN,NaN
23833,342800010008,JHS 8 RICHARD S GROSSLEY,2018,Combined,All Students,669,100.1,670,100,2,NaN,NaN,NaN
9244,261600010004,SCHOOL 4-GEORGE MATHER FORBES,2018,Combined,All Students,612,100.1,613,99.9,2,NaN,NaN,NaN
15281,320700010000,NEW YORK CITY GEOGRAPHIC DISTRICT # 7,2018,Combined,All Students,16128,100.1,16247,99.4,2,NaN,NaN,NaN
39268,661401030005,ROOSEVELT SCHOOL,2018,Combined,All Students,584,100.1,713,82,2,NaN,NaN,NaN
3934,131500010006,G W KRIEGER SCHOOL,2018,Combined,All Students,621,100.2,629,99,2,NaN,NaN,NaN
31060,511301040000,HERMON-DEKALB CENTRAL SCHOOL DISTRICT,2018,Combined,All Students,403,100.2,417,96.9,2,NaN,NaN,NaN
17032,321000010244,NEW SCHOOL FOR LEADERSHIP AND JOURNALISM (THE),2018,Combined,All Students,1613,100.2,1621,99.7,2,NaN,NaN,NaN
24480,342900010195,PS 195 WILLIAM HABERLE,2018,Combined,All Students,547,100.3,547,100.3,2,NaN,NaN,NaN


In [212]:
elem_df.composite_level.value_counts()

2    11674
s    11192
4     7333
3     7318
1     2863
Name: composite_level, dtype: int64

In [213]:
elem_df.subgroup_name.value_counts()

All Students                                       4522
Students with Disabilities                         4509
Economically Disadvantaged                         4477
White                                              4409
Hispanic or Latino                                 4370
Black or African American                          4159
Asian or Native Hawaiian/Other Pacific Islander    3969
Multiracial                                        3823
English Language Learners                          3633
American Indian or Alaska Native                   2509
Name: subgroup_name, dtype: int64

In [214]:
hs_df

,entity_cd,entity_name,year,subject,subgroup_name,composite_cohort,composite__index,composite__level,override
0,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,All Students,2152,146.5,1,NaN
1,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Students with Disabilities,284,79.1,1,NaN
2,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,American Indian or Alaska Native,12,s,s,NaN
3,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Asian or Native Hawaiian/Other Pacific Islander,200,165.6,2,NaN
4,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Black or African American,1052,132.9,1,NaN
5,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Hispanic or Latino,380,138,1,NaN
6,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,White,468,173.9,2,NaN
7,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,Multiracial,104,161.3,2,NaN
8,10100860907,GREEN TECH HIGH CHARTER SCHOOL,2018,Combined,Economically Disadvantaged,212,151.7,2,NaN
9,10100860960,ALBANY LEADERSHIP CHARTER HIGH SCHOOL FOR GIRLS,2018,Combined,All Students,232,158.4,2,NaN


In [215]:
dem_df

,entity_cd,entity_name,year,num_ell,per_ell,num_am_ind,per_am_ind,num_black,per_black,num_hisp,...,num_ecdis,per_ecdis,num_migrant,per_migrant,num_homeless,per_homeless,num_foster,per_foster,num_armed,per_armed
0,3000000,BROOME County,2017,487,2,74,0,2370,9,1693,...,14110,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4000000,CATTARAUGUS County,2017,23,0,776,6,241,2,330,...,7053,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5000000,CAYUGA County,2017,50,1,18,0,308,3,335,...,4609,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6000000,CHAUTAUQUA County,2017,636,3,189,1,396,2,2567,...,10442,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7000000,CHEMUNG County,2017,46,0,23,0,925,8,286,...,5967,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8000000,CHENANGO County,2017,14,0,33,0,103,1,192,...,3689,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,9000000,CLINTON County,2017,29,0,29,0,223,2,118,...,5440,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10000000,COLUMBIA County,2017,293,4,5,0,505,7,608,...,3514,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11000000,CORTLAND County,2017,23,0,3,0,106,2,163,...,3312,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,12000000,DELAWARE County,2017,47,1,14,0,72,1,262,...,2968,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
lunch_df.head()

,entity_cd,entity_name,year,num_free_lunch,per_free_lunch,num_reduced_lunch,per_reduced_lunch
0,1,NYC Public Schools,2018,670164,72,37424,4
1,2,Large Cities,2018,81038,79,1004,1
2,3,High Need/Resource Urban-Suburban Districts,2018,141728,69,8723,4
3,4,High Need/Resource Rural Districts,2018,72007,51,9301,7
4,5,Average Need Districts,2018,241901,33,35802,5


135% - free lunch
185% - reduced lunch

In [217]:
ins_df.head()

,group_code,group_name,entity_cd,entity_name
0,1,Total Public,111111111111,All Public Schools
1,2,County Public Schools,1000000,ALBANY County
2,2,County Public Schools,2000000,ALLEGANY County
3,2,County Public Schools,3000000,BROOME County
4,2,County Public Schools,4000000,CATTARAUGUS County


In [218]:
for a, x in zip(str_list_dfs, list_dfs):
    print (f'Dataframe {a} is {x.shape} in shape')

Dataframe lead_df is (4682, 27) in shape
Dataframe elem_df is (40380, 13) in shape
Dataframe hs_df is (15995, 9) in shape
Dataframe dem_df is (16570, 33) in shape
Dataframe lunch_df is (16570, 7) in shape
Dataframe ins_df is (5522, 4) in shape


lead_df will guide our dataframe set up since it is the limiting factor with the fewest entries. BOCES code will allow us to join the various tables together.

In [219]:
# define function that returns a dataframe with fraction and percent of missing 
# data for each column
def percent_missing (df):
    numer = df.isnull().sum() 
    denom = df.isnull().count()

    data= []
    i = 0
    for x,y in zip(numer,denom):
        data.append([numer.index[i],(Fraction(x,y)), x*100/y])
        i+=1
    
    df1 = pd.DataFrame(data,columns = ['Column_Name', 'Fraction of Missing Data','Percent'])
    return df1

# preview this function in action with our defined dataframe
percent_missing(lead_df)

,Column_Name,Fraction of Missing Data,Percent
0,school_district,0,0.000000
1,school,0,0.000000
2,county,0,0.000000
3,type_of_organization,0,0.000000
4,number_of_outlets,19/2341,0.811619
5,any_buildings_with_lead-free_plumbing?,0,0.000000
6,previously_sampled_outlets,19/2341,0.811619
7,outlets_waiver_requested,19/2341,0.811619
8,waivers_granted,19/2341,0.811619
9,outlets_sampled_after_regulation,19/2341,0.811619


In [220]:
# use above function to return dataframe containing only columns that have some missing data
def missing_data (df):
    missingdf = percent_missing(df)
    missingdf1 = missingdf[missingdf['Fraction of Missing Data'] > 0]
    print(f'{missingdf1.shape[0]} out of {missingdf.shape[0]} columns have missing data')
    return missingdf1

missing_data(lead_df)

16 out of 27 columns have missing data


,Column_Name,Fraction of Missing Data,Percent
4,number_of_outlets,19/2341,0.811619
6,previously_sampled_outlets,19/2341,0.811619
7,outlets_waiver_requested,19/2341,0.811619
8,waivers_granted,19/2341,0.811619
9,outlets_sampled_after_regulation,19/2341,0.811619
11,sampling_completion_date,23/2341,0.982486
12,"number_of_outlets,_result_≤_15_ppb",34/2341,1.452371
13,"number_of_outlets,_result_>_15_ppb",37/2341,1.580521
16,date_all_results_received,81/4682,1.730030
17,school_website,19/2341,0.811619


In [221]:
def num_col (df):
    numeric_columns = df.select_dtypes(['int64','float64']).columns
    print(f"The number of numerical columns is {len(numeric_columns)}")
    return numeric_columns

def non_num_col (df):
    non_numeric_columns = df.select_dtypes(['object']).columns
    print(f"The number of non-numerical columns is {len(non_numeric_columns)}")
    return non_numeric_columns


In [222]:
numeric_columns = num_col(lead_df)

The number of numerical columns is 7


In [223]:
non_numeric_columns = non_num_col(lead_df)

The number of non-numerical columns is 20


In [224]:
lead_df.columns

Index(['school_district', 'school', 'county', 'type_of_organization',
       'number_of_outlets', 'any_buildings_with_lead-free_plumbing?',
       'previously_sampled_outlets', 'outlets_waiver_requested',
       'waivers_granted', 'outlets_sampled_after_regulation',
       'sampling_complete', 'sampling_completion_date',
       'number_of_outlets,_result_≤_15_ppb',
       'number_of_outlets,_result_>_15_ppb', 'out_of_service',
       'all_results_received', 'date_all_results_received', 'school_website',
       'beds_code', 'school_street', 'school_city', 'school_state',
       'school_zip_code', 'date_sampling_updated', 'date_results_updated',
       'county_location', 'location'],
      dtype='object')

In [225]:
minilead_df = lead_df[[
    'beds_code',
    'school_district',
    'school',
    'county',
    'school_city',
    'school_zip_code',
    'number_of_outlets,_result_≤_15_ppb',
    'number_of_outlets,_result_>_15_ppb',
    'any_buildings_with_lead-free_plumbing?']]

In [226]:
minilead_df

,beds_code,school_district,school,county,school_city,school_zip_code,"number_of_outlets,_result_≤_15_ppb","number_of_outlets,_result_>_15_ppb",any_buildings_with_lead-free_plumbing?
0,599000000000,BOCES No District Name,SULLIVAN BOCES,Sullivan,LIBERTY,12754,NaN,NaN,No
1,649000000006,BOCES No District Name,WSWHE BOCES MEYERS EDUCATIONAL CENTER MAIN BUI...,Washington,SARATOGA SPRINGS,12866,147.0,48.0,No
2,141601060007,HAMBURG CENTRAL SCHOOL,HAMBURG SHS,Erie,HAMBURG,14075,98.0,6.0,No
3,650501040001,LYONS CENTRAL SCHOOL,LYONS ELEMENTARY SCHOOL,Wayne,LYONS,14489,27.0,2.0,No
4,450801060002,MEDINA CENTRAL SCHOOL,OAK ORCHARD SCHOOL,Orleans,MEDINA,14103,110.0,5.0,No
5,411501060004,NEW HARTFORD CENTRAL SCHOOL,BRADLEY ELEMENTARY SCHOOL,Oneida,NEW HARTFORD,13413,97.0,0.0,No
6,141101060002,SPRINGVILLE-GRIFF INST CENTRAL SCHOOL,COLDEN ES,Erie,COLDEN,14033,55.0,6.0,No
7,200901040001,WELLS CENTRAL SCHOOL,WELLS SCHOOL,Hamilton,WELLS,12190,44.0,11.0,No
8,X085,NYC DOE,X085,Bronx,Bronx,10458,78.0,0.0,No
9,600301040002,CANDOR CENTRAL SCHOOL,CANDOR J S H S,Tioga,CANDOR,13743,116.0,0.0,No


## 4. Feature Engineering

In [227]:
# we want to merge beds_code from minilead_df with entity_cd from the other tables
# let's check the type of entry for these columns

minilead_df.dtypes

beds_code                                  object
school_district                            object
school                                     object
county                                     object
school_city                                object
school_zip_code                            object
number_of_outlets,_result_≤_15_ppb        float64
number_of_outlets,_result_>_15_ppb        float64
any_buildings_with_lead-free_plumbing?     object
dtype: object

In [228]:
minilead_df

,beds_code,school_district,school,county,school_city,school_zip_code,"number_of_outlets,_result_≤_15_ppb","number_of_outlets,_result_>_15_ppb",any_buildings_with_lead-free_plumbing?
0,599000000000,BOCES No District Name,SULLIVAN BOCES,Sullivan,LIBERTY,12754,NaN,NaN,No
1,649000000006,BOCES No District Name,WSWHE BOCES MEYERS EDUCATIONAL CENTER MAIN BUI...,Washington,SARATOGA SPRINGS,12866,147.0,48.0,No
2,141601060007,HAMBURG CENTRAL SCHOOL,HAMBURG SHS,Erie,HAMBURG,14075,98.0,6.0,No
3,650501040001,LYONS CENTRAL SCHOOL,LYONS ELEMENTARY SCHOOL,Wayne,LYONS,14489,27.0,2.0,No
4,450801060002,MEDINA CENTRAL SCHOOL,OAK ORCHARD SCHOOL,Orleans,MEDINA,14103,110.0,5.0,No
5,411501060004,NEW HARTFORD CENTRAL SCHOOL,BRADLEY ELEMENTARY SCHOOL,Oneida,NEW HARTFORD,13413,97.0,0.0,No
6,141101060002,SPRINGVILLE-GRIFF INST CENTRAL SCHOOL,COLDEN ES,Erie,COLDEN,14033,55.0,6.0,No
7,200901040001,WELLS CENTRAL SCHOOL,WELLS SCHOOL,Hamilton,WELLS,12190,44.0,11.0,No
8,X085,NYC DOE,X085,Bronx,Bronx,10458,78.0,0.0,No
9,600301040002,CANDOR CENTRAL SCHOOL,CANDOR J S H S,Tioga,CANDOR,13743,116.0,0.0,No


In [229]:
# check entity_cd type in dem_df
dem_df.dtypes

entity_cd         int64
entity_name      object
year              int64
num_ell           int64
per_ell           int64
num_am_ind        int64
per_am_ind        int64
num_black         int64
per_black         int64
num_hisp          int64
per_hisp          int64
num_asian         int64
per_asian         int64
num_white         int64
per_white         int64
num_multi         int64
per_multi         int64
num_swd           int64
per_swd           int64
num_female        int64
per_female        int64
num_male          int64
per_male          int64
num_ecdis         int64
per_ecdis         int64
num_migrant     float64
per_migrant     float64
num_homeless    float64
per_homeless    float64
num_foster      float64
per_foster      float64
num_armed       float64
per_armed       float64
dtype: object

In [230]:
# to merge, we need these columns to be the same type. It's easier to make the integers 
# into strings since our beds_code has some strings that aren't strictly numbers.

dem_df1 = dem_df.copy()
dem_df1 = dem_df1[dem_df1['year'] == 2018]
dem_df1['entity_cd'] = dem_df1['entity_cd'].apply(str)

In [231]:
# let's verify the change
dem_df1.dtypes

entity_cd        object
entity_name      object
year              int64
num_ell           int64
per_ell           int64
num_am_ind        int64
per_am_ind        int64
num_black         int64
per_black         int64
num_hisp          int64
per_hisp          int64
num_asian         int64
per_asian         int64
num_white         int64
per_white         int64
num_multi         int64
per_multi         int64
num_swd           int64
per_swd           int64
num_female        int64
per_female        int64
num_male          int64
per_male          int64
num_ecdis         int64
per_ecdis         int64
num_migrant     float64
per_migrant     float64
num_homeless    float64
per_homeless    float64
num_foster      float64
per_foster      float64
num_armed       float64
per_armed       float64
dtype: object

In [232]:
# verify how the dataframe looks
dem_df1

,entity_cd,entity_name,year,num_ell,per_ell,num_am_ind,per_am_ind,num_black,per_black,num_hisp,...,num_ecdis,per_ecdis,num_migrant,per_migrant,num_homeless,per_homeless,num_foster,per_foster,num_armed,per_armed
11055,1,NYC Public Schools,2018,142556,15,9994,1,214312,23,383293,...,707879,76,0.0,0.0,88268.0,9.0,0.0,0.0,440.0,0.0
11056,2,Large Cities,2018,16421,16,540,1,43641,43,30892,...,86527,85,78.0,0.0,3693.0,4.0,65.0,0.0,7.0,0.0
11057,3,High Need/Resource Urban-Suburban Districts,2018,34135,17,622,0,49593,24,83512,...,160517,78,178.0,0.0,9643.0,5.0,473.0,0.0,564.0,0.0
11058,4,High Need/Resource Rural Districts,2018,1944,1,2333,2,4617,3,10357,...,86074,61,715.0,1.0,2733.0,2.0,472.0,0.0,3770.0,3.0
11059,480101060000,MAHOPAC CENTRAL SCHOOL DISTRICT,2018,134,3,3,0,68,2,785,...,745,18,0.0,0.0,21.0,1.0,0.0,0.0,0.0,0.0
11060,480101060001,MAHOPAC HIGH SCHOOL,2018,23,2,1,0,26,2,236,...,234,16,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
11061,480101060002,AUSTIN ROAD ELEMENTARY SCHOOL,2018,28,5,0,0,12,2,124,...,115,19,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0
11062,480101060004,MAHOPAC MIDDLE SCHOOL,2018,19,2,0,0,17,2,181,...,171,18,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
11063,480101060005,LAKEVIEW ELEMENTARY SCHOOL,2018,52,9,1,0,9,2,155,...,146,25,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0
11064,480101060006,FULMAR ROAD ELEMENTARY SCHOOL,2018,12,2,1,0,4,1,89,...,79,16,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0


In [261]:
dem_df1.columns

Index(['entity_cd', 'entity_name', 'year', 'num_ell', 'per_ell', 'num_am_ind',
       'per_am_ind', 'num_black', 'per_black', 'num_hisp', 'per_hisp',
       'num_asian', 'per_asian', 'num_white', 'per_white', 'num_multi',
       'per_multi', 'num_swd', 'per_swd', 'num_female', 'per_female',
       'num_male', 'per_male', 'num_ecdis', 'per_ecdis', 'num_migrant',
       'per_migrant', 'num_homeless', 'per_homeless', 'num_foster',
       'per_foster', 'num_armed', 'per_armed'],
      dtype='object')

correlation matrix

In [254]:
# merge minilead_df with dem_df1 on the BOCES code.
comp_df = pd.merge(
    minilead_df, 
    dem_df1, 
    left_on='beds_code', 
    right_on='entity_cd',
    how = 'inner')


# look into why half the records fall out

In [256]:
# let's see how it looks
comp_df

,beds_code,school_district,school,county,school_city,school_zip_code,"number_of_outlets,_result_≤_15_ppb","number_of_outlets,_result_>_15_ppb",any_buildings_with_lead-free_plumbing?,entity_cd,...,num_ecdis,per_ecdis,num_migrant,per_migrant,num_homeless,per_homeless,num_foster,per_foster,num_armed,per_armed
0,141601060007,HAMBURG CENTRAL SCHOOL,HAMBURG SHS,Erie,HAMBURG,14075,98.0,6.0,No,141601060007,...,249,22,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0
1,650501040001,LYONS CENTRAL SCHOOL,LYONS ELEMENTARY SCHOOL,Wayne,LYONS,14489,27.0,2.0,No,650501040001,...,352,70,6.0,1.0,23.0,5.0,0.0,0.0,0.0,0.0
2,450801060002,MEDINA CENTRAL SCHOOL,OAK ORCHARD SCHOOL,Orleans,MEDINA,14103,110.0,5.0,No,450801060002,...,268,64,13.0,3.0,5.0,1.0,6.0,1.0,0.0,0.0
3,411501060004,NEW HARTFORD CENTRAL SCHOOL,BRADLEY ELEMENTARY SCHOOL,Oneida,NEW HARTFORD,13413,97.0,0.0,No,411501060004,...,65,14,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,141101060002,SPRINGVILLE-GRIFF INST CENTRAL SCHOOL,COLDEN ES,Erie,COLDEN,14033,55.0,6.0,No,141101060002,...,68,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,200901040001,WELLS CENTRAL SCHOOL,WELLS SCHOOL,Hamilton,WELLS,12190,44.0,11.0,No,200901040001,...,71,51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,600301040002,CANDOR CENTRAL SCHOOL,CANDOR J S H S,Tioga,CANDOR,13743,116.0,0.0,No,600301040002,...,182,52,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0
7,131601060015,ARLINGTON CENTRAL SCHOOL DISTRICT,UNION VALE MIDDLE SCHOOL,Dutchess,LAGRANGEVILLE,12540,150.0,45.0,No,131601060015,...,189,18,0.0,0.0,11.0,1.0,3.0,0.0,1.0,0.0
8,660203060007,HENDRICK HUDSON CENTRAL SCHOOL,BLUE MOUNTAIN MIDDLE SCHOOL,Westchester,CORTLANDT MANOR,10567,31.0,2.0,No,660203060007,...,167,30,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0
9,580212060001,LONGWOOD CENTRAL SCHOOL DISTRICT,C E WALTERS SCHOOL,Suffolk,YAPHANK,11980,202.0,1.0,No,580212060001,...,423,57,1.0,0.0,13.0,2.0,0.0,0.0,0.0,0.0


In [235]:
allelem_df = elem_df[elem_df['subgroup_name']=='All Students']
allelem_df1 = allelem_df.copy()
allelem_df1 = allelem_df1[allelem_df1['year'] == 2018]
allelem_df1['entity_cd'] = allelem_df1['entity_cd'].apply(str)
allelem_df1

,entity_cd,entity_name,year,subject,subgroup_name,core_cohort,core_index,weighted_cohort,weighted_index,composite_level,override,override_weighted,override_core
0,10100010000,ALBANY CITY SCHOOL DISTRICT,2018,Combined,All Students,7998,84.9,9112,74.5,1,NaN,NaN,NaN
10,10100010014,MONTESSORI MAGNET SCHOOL,2018,Combined,All Students,263,166.2,307,142.3,3,NaN,NaN,NaN
20,10100010016,PINE HILLS ELEMENTARY SCHOOL,2018,Combined,All Students,403,109.2,403,109.2,2,NaN,NaN,NaN
30,10100010018,DELAWARE COMMUNITY SCHOOL,2018,Combined,All Students,429,81.5,445,78.5,1,NaN,NaN,NaN
40,10100010019,NEW SCOTLAND ELEMENTARY SCHOOL,2018,Combined,All Students,482,129.1,558,111.6,2,NaN,NaN,NaN
50,10100010020,NORTH ALBANY ACADEMY,2018,Combined,All Students,332,41.6,350,39.4,1,NaN,NaN,NaN
59,10100010023,ALBANY SCHOOL OF HUMANITIES,2018,Combined,All Students,581,105.8,607,101.2,2,NaN,NaN,NaN
69,10100010027,EAGLE POINT ELEMENTARY SCHOOL,2018,Combined,All Students,312,99.8,364,85.6,2,NaN,NaN,NaN
79,10100010028,THOMAS S O'BRIEN ACADEMY OF SCIENCE & TECHNOLOGY,2018,Combined,All Students,523,83.3,523,83.3,1,NaN,NaN,NaN
89,10100010029,GIFFEN MEMORIAL ELEMENTARY SCHOOL,2018,Combined,All Students,575,55.1,575,55.1,1,NaN,NaN,NaN


In [248]:
allelem_df1.composite_level.value_counts()

2    1753
3    1104
4    1100
1     428
s     137
Name: composite_level, dtype: int64

In [236]:
allhs_df = hs_df[hs_df['subgroup_name']=='All Students']
allhs_df1 = allhs_df.copy()
allhs_df1 = allhs_df1[allhs_df1['year'] == 2018]
allhs_df1['entity_cd'] = allhs_df1['entity_cd'].apply(str)
   

In [250]:
allhs_df1.composite__level.value_counts()

2    773
4    483
3    483
1    181
s     23
Name: composite__level, dtype: int64

In [242]:
allhs_df1.dtypes

entity_cd           object
entity_name         object
year                 int64
subject             object
subgroup_name       object
composite_cohort    object
composite__index    object
composite__level    object
override            object
dtype: object

In [243]:
allelem_df1.dtypes

entity_cd            object
entity_name          object
year                  int64
subject              object
subgroup_name        object
core_cohort          object
core_index           object
weighted_cohort      object
weighted_index       object
composite_level      object
override             object
override_weighted    object
override_core        object
dtype: object

In [251]:
allhs_df1.rename(columns={'composite__index':'composite_index',
                          'composite__level':'composite_level',
                          'composite__level':'composite_level',
                          'composite__level':'composite_level',}, inplace=True)

In [ ]:
# proof by inspection

In [259]:
allschools_df = pd.concat([allhs_df1, allelem_df1], ignore_index=True)
allschools_df

,composite_cohort,composite_index,composite_level,core_cohort,core_index,entity_cd,entity_name,override,override_core,override_weighted,subgroup_name,subject,weighted_cohort,weighted_index,year
0,2152,146.5,1,NaN,NaN,10100010000,ALBANY CITY SCHOOL DISTRICT,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
1,232,158.4,2,NaN,NaN,10100860960,ALBANY LEADERSHIP CHARTER HIGH SCHOOL FOR GIRLS,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
2,300,185.5,3,NaN,NaN,10201040000,BERNE-KNOX-WESTERLO CENTRAL SCHOOL DISTRICT,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
3,2112,147.9,2,NaN,NaN,10100010034,ALBANY HIGH SCHOOL,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
4,188,161.1,2,NaN,NaN,10100860907,GREEN TECH HIGH CHARTER SCHOOL,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
5,536,170.9,2,NaN,NaN,10402060000,RAVENA-COEYMANS-SELKIRK CENTRAL SCHOOL DISTRICT,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
6,292,187.1,3,NaN,NaN,10201040001,BERNE-KNOX-WESTERLO JUNIOR-SENIOR HIGH SCHOOL,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
7,1668,214.4,4,NaN,NaN,10306060000,BETHLEHEM CENTRAL SCHOOL DISTRICT,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
8,1644,216.5,4,NaN,NaN,10306060008,BETHLEHEM CENTRAL SENIOR HIGH SCHOOL,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018
9,520,173.1,3,NaN,NaN,10402060001,RAVENA-COEYMANS-SELKIRK SENIOR HIGH SCHOOL,NaN,NaN,NaN,All Students,Combined,NaN,NaN,2018


In [183]:
# merge comp_df with allelem_df1 on the BOCES code.
comp_df1 = pd.merge(
    comp_df, 
    allelem_df1, 
    left_on='beds_code', 
    right_on='entity_cd',
    how = 'left')
comp_df1

,beds_code,school_district,school,county,school_city,school_zip_code,"number_of_outlets,_result_≤_15_ppb","number_of_outlets,_result_>_15_ppb",any_buildings_with_lead-free_plumbing?,entity_cd_x,...,subject,subgroup_name,core_cohort,core_index,weighted_cohort,weighted_index,composite_level,override,override_weighted,override_core
0,599000000000,BOCES No District Name,SULLIVAN BOCES,Sullivan,LIBERTY,12754,NaN,NaN,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,649000000006,BOCES No District Name,WSWHE BOCES MEYERS EDUCATIONAL CENTER MAIN BUI...,Washington,SARATOGA SPRINGS,12866,147.0,48.0,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141601060007,HAMBURG CENTRAL SCHOOL,HAMBURG SHS,Erie,HAMBURG,14075,98.0,6.0,No,141601060007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,650501040001,LYONS CENTRAL SCHOOL,LYONS ELEMENTARY SCHOOL,Wayne,LYONS,14489,27.0,2.0,No,650501040001,...,Combined,All Students,607,79,611,78.5,1,NaN,NaN,NaN
4,450801060002,MEDINA CENTRAL SCHOOL,OAK ORCHARD SCHOOL,Orleans,MEDINA,14103,110.0,5.0,No,450801060002,...,Combined,All Students,198,118.4,207,113.3,2,NaN,NaN,NaN
5,411501060004,NEW HARTFORD CENTRAL SCHOOL,BRADLEY ELEMENTARY SCHOOL,Oneida,NEW HARTFORD,13413,97.0,0.0,No,411501060004,...,Combined,All Students,368,187.8,601,115,3,NaN,NaN,NaN
6,141101060002,SPRINGVILLE-GRIFF INST CENTRAL SCHOOL,COLDEN ES,Erie,COLDEN,14033,55.0,6.0,No,141101060002,...,Combined,All Students,204,149.5,252,121,3,NaN,NaN,NaN
7,200901040001,WELLS CENTRAL SCHOOL,WELLS SCHOOL,Hamilton,WELLS,12190,44.0,11.0,No,200901040001,...,Combined,All Students,144,138.9,144,138.9,3,NaN,NaN,NaN
8,X085,NYC DOE,X085,Bronx,Bronx,10458,78.0,0.0,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,600301040002,CANDOR CENTRAL SCHOOL,CANDOR J S H S,Tioga,CANDOR,13743,116.0,0.0,No,600301040002,...,Combined,All Students,250,105.8,295,89.7,2,NaN,NaN,NaN


In [189]:
# merge comp_df1 with allhs_df1 on the BOCES code.
comp_df2 = pd.merge(
    comp_df1, 
    allhs_df1, 
    left_on='beds_code', 
    right_on='entity_cd',
    how = 'left')
comp_df2

,beds_code,school_district,school,county,school_city,school_zip_code,"number_of_outlets,_result_≤_15_ppb","number_of_outlets,_result_>_15_ppb",any_buildings_with_lead-free_plumbing?,entity_cd_x,...,override_core,entity_cd,entity_name,year,subject_y,subgroup_name_y,composite_cohort,composite__index,composite__level,override_y
0,599000000000,BOCES No District Name,SULLIVAN BOCES,Sullivan,LIBERTY,12754,NaN,NaN,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,649000000006,BOCES No District Name,WSWHE BOCES MEYERS EDUCATIONAL CENTER MAIN BUI...,Washington,SARATOGA SPRINGS,12866,147.0,48.0,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141601060007,HAMBURG CENTRAL SCHOOL,HAMBURG SHS,Erie,HAMBURG,14075,98.0,6.0,No,141601060007,...,NaN,141601060007,HAMBURG HIGH SCHOOL,2018.0,Combined,All Students,1068,212.7,4,NaN
3,650501040001,LYONS CENTRAL SCHOOL,LYONS ELEMENTARY SCHOOL,Wayne,LYONS,14489,27.0,2.0,No,650501040001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,450801060002,MEDINA CENTRAL SCHOOL,OAK ORCHARD SCHOOL,Orleans,MEDINA,14103,110.0,5.0,No,450801060002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,411501060004,NEW HARTFORD CENTRAL SCHOOL,BRADLEY ELEMENTARY SCHOOL,Oneida,NEW HARTFORD,13413,97.0,0.0,No,411501060004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,141101060002,SPRINGVILLE-GRIFF INST CENTRAL SCHOOL,COLDEN ES,Erie,COLDEN,14033,55.0,6.0,No,141101060002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,200901040001,WELLS CENTRAL SCHOOL,WELLS SCHOOL,Hamilton,WELLS,12190,44.0,11.0,No,200901040001,...,NaN,200901040001,WELLS SCHOOL,2018.0,Combined,All Students,112,171.6,2,NaN
8,X085,NYC DOE,X085,Bronx,Bronx,10458,78.0,0.0,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,600301040002,CANDOR CENTRAL SCHOOL,CANDOR J S H S,Tioga,CANDOR,13743,116.0,0.0,No,600301040002,...,NaN,600301040002,CANDOR JUNIOR-SENIOR HIGH SCHOOL,2018.0,Combined,All Students,176,154.7,2,NaN


In [177]:
lunch_df1 = lunch_df.copy()
lunch_df1 = lunch_df1[lunch_df1['year'] == 2018]
lunch_df1['entity_cd'] = lunch_df1['entity_cd'].apply(str)
lunch_df1

,entity_cd,entity_name,year,num_free_lunch,per_free_lunch,num_reduced_lunch,per_reduced_lunch
0,1,NYC Public Schools,2018,670164,72,37424,4
1,2,Large Cities,2018,81038,79,1004,1
2,3,High Need/Resource Urban-Suburban Districts,2018,141728,69,8723,4
3,4,High Need/Resource Rural Districts,2018,72007,51,9301,7
4,5,Average Need Districts,2018,241901,33,35802,5
5,6,Low Need Districts,2018,44433,12,7061,2
6,7,Charter Schools,2018,102251,73,9352,7
7,1000000,ALBANY County,2018,15258,39,1014,3
8,2000000,ALLEGANY County,2018,2840,47,408,7
9,3000000,BROOME County,2018,12602,49,1108,4


In [185]:
# merge comp_df2 with lunch_df1 on the BOCES code.
comp_df3 = pd.merge(
    comp_df2, 
    lunch_df1, 
    left_on='beds_code', 
    right_on='entity_cd',
    how = 'left')
comp_df3

,beds_code,school_district,school,county,school_city,school_zip_code,"number_of_outlets,_result_≤_15_ppb","number_of_outlets,_result_>_15_ppb",any_buildings_with_lead-free_plumbing?,entity_cd_x,...,composite__index,composite__level,override_y,entity_cd_y,entity_name_y,year_y,num_free_lunch,per_free_lunch,num_reduced_lunch,per_reduced_lunch
0,599000000000,BOCES No District Name,SULLIVAN BOCES,Sullivan,LIBERTY,12754,NaN,NaN,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,649000000006,BOCES No District Name,WSWHE BOCES MEYERS EDUCATIONAL CENTER MAIN BUI...,Washington,SARATOGA SPRINGS,12866,147.0,48.0,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141601060007,HAMBURG CENTRAL SCHOOL,HAMBURG SHS,Erie,HAMBURG,14075,98.0,6.0,No,141601060007,...,212.7,4,NaN,141601060007,HAMBURG HIGH SCHOOL,2018.0,187.0,17.0,45.0,4.0
3,650501040001,LYONS CENTRAL SCHOOL,LYONS ELEMENTARY SCHOOL,Wayne,LYONS,14489,27.0,2.0,No,650501040001,...,NaN,NaN,NaN,650501040001,LYONS ELEMENTARY SCHOOL,2018.0,294.0,59.0,26.0,5.0
4,450801060002,MEDINA CENTRAL SCHOOL,OAK ORCHARD SCHOOL,Orleans,MEDINA,14103,110.0,5.0,No,450801060002,...,NaN,NaN,NaN,450801060002,OAK ORCHARD SCHOOL,2018.0,232.0,56.0,29.0,7.0
5,411501060004,NEW HARTFORD CENTRAL SCHOOL,BRADLEY ELEMENTARY SCHOOL,Oneida,NEW HARTFORD,13413,97.0,0.0,No,411501060004,...,NaN,NaN,NaN,411501060004,ROBERT L BRADLEY ELEMENTARY SCHOOL,2018.0,38.0,8.0,12.0,3.0
6,141101060002,SPRINGVILLE-GRIFF INST CENTRAL SCHOOL,COLDEN ES,Erie,COLDEN,14033,55.0,6.0,No,141101060002,...,NaN,NaN,NaN,141101060002,COLDEN ELEMENTARY SCHOOL,2018.0,46.0,25.0,10.0,5.0
7,200901040001,WELLS CENTRAL SCHOOL,WELLS SCHOOL,Hamilton,WELLS,12190,44.0,11.0,No,200901040001,...,171.6,2,NaN,200901040001,WELLS SCHOOL,2018.0,55.0,40.0,14.0,10.0
8,X085,NYC DOE,X085,Bronx,Bronx,10458,78.0,0.0,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,600301040002,CANDOR CENTRAL SCHOOL,CANDOR J S H S,Tioga,CANDOR,13743,116.0,0.0,No,600301040002,...,154.7,2,NaN,600301040002,CANDOR JUNIOR-SENIOR HIGH SCHOOL,2018.0,146.0,42.0,20.0,6.0


In [ ]:
# 1) masterlist of schools
# 2) demographics
# 3) lunch data
# 4) lead (limit, right)

## 5. Exploratory data analysis

## 6. Supervised Learning Models

## 7. Summary of Results

### Pros and Cons of Dataset:

### Future Study:
